# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
import random
from dataclasses import dataclass
from itertools import product
from typing import Union, List, Tuple

import numpy as np
import pandas
from catboost import CatBoostClassifier
from scipy.stats import mode
from sklearn.model_selection import train_test_split, KFold
from tqdm.notebook import tqdm

In [2]:
SEED = 7
np.random.seed(SEED)
random.seed(SEED)

In [3]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [4]:
@dataclass
class DecisionTreeLeaf:
    classes: np.ndarray

    def __post_init__(self):
        self.max_class = mode(self.classes)[0][0]

@dataclass
class DecisionTreeInternalNode:
    split_dim: int
    left: Union['DecisionTreeInternalNode', DecisionTreeLeaf]
    right: Union['DecisionTreeInternalNode', DecisionTreeLeaf]


DecisionTreeNode = Union[DecisionTreeInternalNode, DecisionTreeLeaf]

In [5]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        if criterion == "gini":
            self._criterion = gini
        elif criterion == "entropy":
            self._criterion = entropy
        else:
            raise ValueError(f"Unknown criterion function: {criterion}")

        if isinstance(max_features, int):
            self._max_features = max_features
        elif max_features == "auto":
            self._max_features = int(np.sqrt(X.shape[1]))
        else:
            raise ValueError(f"Unknown value of max features parameter: {max_features}")

        self._max_depth = max_depth
        self._min_samples_leaf = min_samples_leaf

        self._bagged_idx = np.random.choice(X.shape[0], X.shape[0])
        self._bagged_X, self._bagged_y = X[self._bagged_idx], y[self._bagged_idx]

        mask = np.zeros(X.shape[0], dtype=np.bool)
        mask[self._bagged_idx] = True
        self._out_of_bag_idx = np.argwhere(~mask).reshape(-1)
        self._out_of_bag_X, self._out_of_bag_y = X[self._out_of_bag_idx], y[self._out_of_bag_idx]

        self._root = self._build_node(self._bagged_X, self._bagged_y, 0)

    @property
    def out_of_bag(self) -> Tuple[np.ndarray, np.ndarray]:
        return self._out_of_bag_X, self._out_of_bag_y

    def _build_node(self, points: np.ndarray, classes: np.ndarray, depth: int) -> DecisionTreeNode:
        if self._max_depth is not None and depth >= self._max_depth:
            return DecisionTreeLeaf(classes)

        cur_features = np.random.choice(points.shape[1], self._max_features, replace=False)

        n_points, n_features = points.shape
        split_dim, split_gain = None, 0.0
        for cur_dim in cur_features:
            mask = points[:, cur_dim] == 1

            left_size = mask.sum()
            right_size = n_points - left_size
            if left_size < self._min_samples_leaf or right_size < self._min_samples_leaf:
                continue

            cur_gain = gain(classes[mask], classes[~mask], self._criterion)
            if cur_gain > split_gain:
                split_dim, split_gain = cur_dim, cur_gain

        if split_dim is None:
            return DecisionTreeLeaf(classes)
        mask = points[:, split_dim] == 1
        left_child = self._build_node(points[mask], classes[mask], depth + 1)
        right_child = self._build_node(points[~mask], classes[~mask], depth + 1)
        return DecisionTreeInternalNode(split_dim, left_child, right_child)

    def _predict(self, points: np.ndarray, node: DecisionTreeNode) -> np.ndarray:
        if isinstance(node, DecisionTreeLeaf):
            return np.full(points.shape[0], node.max_class)
        mask = points[:, node.split_dim] == 1
        result = np.empty(points.shape[0], dtype=np.object)
        result[mask] = self._predict(points[mask], node.left)
        result[~mask] = self._predict(points[~mask], node.right)
        return result

    def predict(self, points: np.ndarray) -> np.ndarray:
        return self._predict(points, self._root)

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [6]:
class RandomForestClassifier:

    _n_features: int = None

    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self._criterion = criterion
        self._max_depth = max_depth
        self._min_samples_leaf = min_samples_leaf
        self._max_features = max_features
        self._n_estimators = n_estimators
        self._estimators = []

    @property
    def estimators(self) -> List[DecisionTree]:
        return self._estimators

    @property
    def n_features(self) -> int:
        if self._n_features is None:
            raise RuntimeError("Fit random forest before accessing to number of features properties")
        return self._n_features

    def fit(self, X, y):
        self._n_features = X.shape[1]
        self._estimators = [
            DecisionTree(X, y, self._criterion, self._max_depth, self._min_samples_leaf, self._max_features)
            for _ in range(self._n_estimators)
        ]
    
    def predict(self, X):
        predictions = np.stack([est.predict(X) for est in tqdm(self._estimators)], axis=0)
        return mode(predictions, axis=0)[0][0]

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [7]:
def accuracy_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    return np.mean(y_true == y_pred)


def feature_importance(rfc):
    estimators = rfc.estimators
    err_oob = np.zeros((len(estimators), 1))
    feature_err_oob = np.zeros((len(estimators), rfc.n_features))
    for i, est in tqdm(enumerate(rfc.estimators), total=len(estimators)):
        X, y = est.out_of_bag
        err_oob[i] = accuracy_score(y, est.predict(X))
        for feature in range(rfc.n_features):
            shuffled_x = X.copy()
            shuffled_x[:, feature] = np.random.permutation(X[:, feature])
            feature_err_oob[i, feature] = accuracy_score(y, est.predict(shuffled_x))
    feature_importance = err_oob - feature_err_oob
    return np.mean(feature_importance, axis=0)


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    indices = np.argsort(importance)[::-1][:k]
    return np.array(names)[indices]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [8]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy: 1.0


  0%|          | 0/100 [00:00<?, ?it/s]

Importance: [-1.11859299e-03  1.14130634e-04  1.68497018e-01  1.75751831e-01
  3.15976483e-01  1.12490697e-03]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [9]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [10]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [11]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

  0%|          | 0/10 [00:00<?, ?it/s]

Accuracy: 0.7061790668348046
Most important features:


  0%|          | 0/10 [00:00<?, ?it/s]

1. ovsyanochan
2. 4ch
3. rhymes
4. styd.pozor
5. mudakoff
6. dayvinchik
7. rapnewrap
8. iwantyou
9. tumblr_vacuum
10. bot_maxim
11. pravdashowtop
12. ultrapir
13. leprum
14. pixel_stickers
15. reflexia_our_feelings
16. ne1party
17. pozor
18. thesmolny
19. memeboizz
20. in.humour


#### Пол

In [12]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

  0%|          | 0/10 [00:00<?, ?it/s]

Accuracy: 0.8474148802017655
Most important features:


  0%|          | 0/10 [00:00<?, ?it/s]

1. 40kg
2. mudakoff
3. 9o_6o_9o
4. modnailru
5. girlmeme
6. bon
7. be.women
8. rapnewrap
9. be.beauty
10. i_d_t
11. thesmolny
12. femalemem
13. zerofat
14. igm
15. beauty
16. soverwenstvo.decora
17. cook_good
18. woman.blog
19. academyofman
20. reflexia_our_feelings


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [13]:
X, y = synthetic_dataset(1000)

cb_model = CatBoostClassifier(loss_function="MultiClass", verbose=False, random_state=SEED)
cb_model.fit(X, y)
y_pred = cb_model.predict(X)

print("Accuracy:", accuracy_score(y_pred, y))
print("Importance:", cb_model.feature_importances_)

Accuracy: 1.0
Importance: [1.02868859e-02 5.15331135e-03 2.78522175e+01 2.79007226e+01
 4.42293440e+01 2.27569188e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [14]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

In [15]:
max_depth = range(1, 10, 3)
min_samples_leaf = range(1, 10, 3)
learning_rate = np.linspace(0.001, 1.0, 5)

def get_best_params(y_train, y_eval):
    best_score, best_params = None, None
    for lr, md, msl in tqdm(list(product(learning_rate, max_depth, min_samples_leaf))):

        classifier = CatBoostClassifier(
            loss_function="MultiClass", verbose=False, random_state=SEED, max_depth=md, min_data_in_leaf=msl, learning_rate=lr
        )
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_eval)
        score = accuracy_score(y_eval, y_pred)

        if best_score is None or score > best_score:
            best_score = score
            best_params = (lr, md, msl)
    return best_params, best_score

#### Возраст

In [16]:
best_params, best_score = get_best_params(y_age_train, y_age_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

((0.25075, 7, 1), 0.7505255781359496)

In [17]:
cb_model = CatBoostClassifier(
    loss_function="MultiClass",
    verbose=False,
    random_state=SEED,
    learning_rate=best_params[0],
    max_depth=best_params[1],
    min_data_in_leaf=best_params[2],
)
cb_model.fit(X_train, y_age_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_age_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.755359394703657
Most important features:
1. mudakoff
2. 4ch
3. ovsyanochan
4. kino_mania
5. rhymes
6. dayvinchik
7. exclusive_muzic
8. rapnewrap
9. styd.pozor
10. kinomania


#### Пол

In [18]:
best_params, best_score = get_best_params(y_sex_train, y_sex_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

((0.5005, 4, 1), 0.8710581639803784)

In [19]:
cb_model = CatBoostClassifier(
    loss_function="MultiClass",
    verbose=False,
    random_state=SEED,
    learning_rate=best_params[0],
    max_depth=best_params[1],
    min_data_in_leaf=best_params[2],
)
cb_model.fit(X_train, y_sex_train)
y_pred = cb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_sex_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8726355611601513
Most important features:
1. 40kg
2. mudakoff
3. modnailru
4. girlmeme
5. 9o_6o_9o
6. i_d_t
7. 4ch
8. rapnewrap
9. fuck_humor
10. team
